Part I. Association Rule Mining

In [1]:
!pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=662c8b2dd2d985c75b9dc75bab6cda4fe73d46511100f58aba32c8fc9e4a6244
  Stored in directory: /root/.cache/pip/wheels/7f/49/e3/42c73b19a264de37129fadaa0c52f26cf50e87de08fb9804af
Successfully built apyori


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori
# Pycaret no longer supports arules
# from pycaret.arules import *

store_data = pd.read_csv('transactions.csv') # Removed header=None
print(store_data.head())
print(store_data.shape)

   transaction_id                                             item_1  \
0               1  Honeywell 5800CO Carbon Monoxide Detector (SKU...   
1               2   Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)   
2               3         Bosch F220-B6 Detector Base (SKU: F220-B6)   
3               4  GE Interlogix 60-652-95R Carbon Monoxide Detec...   
4               5   Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)   

                                              item_2  \
0                Hanwha XRN-2010 NVR (SKU: XRN-2010)   
1             Bosch B5512 Control Panel (SKU: B5512)   
2  GE Interlogix 60-652-95R Carbon Monoxide Detec...   
3         Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2)   
4       Pelco DSSRV2-040-US NVR (SKU: DSSRV2-040-US)   

                                              item_3  \
0                                                NaN   
1     Pelco IMP1110-1ES IP Camera (SKU: IMP1110-1ES)   
2  DSC WS4933 Carbon Monoxide Detector (SKU: WS4933)   
3   Bo

In [3]:
# Build the transaction list
item_cols = [c for c in store_data.columns if c.startswith("item_")]

transactions = []

for _, row in store_data.iterrows():
    # Extract item names from the row
    items = [str(row[c]).strip() for c in item_cols if pd.notna(row[c]) and str(row[c]).strip() != ""]

    # Deduplicate while preserving order
    seen = set()
    clean_items = []
    for i in items:
        if i not in seen:
            seen.add(i)
            clean_items.append(i)

    # Append the cleaned list if it’s not empty
    if clean_items:
        transactions.append(clean_items)

# --- 4) Inspect the result ---
print(f"✅ Built {len(transactions)} transactions.")
print("First few transactions:")
for t in transactions[:10]:
    print(t)

✅ Built 1000 transactions.
First few transactions:
['Honeywell 5800CO Carbon Monoxide Detector (SKU: 5800CO)', 'Hanwha XRN-2010 NVR (SKU: XRN-2010)']
['Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)', 'Bosch B5512 Control Panel (SKU: B5512)', 'Pelco IMP1110-1ES IP Camera (SKU: IMP1110-1ES)', 'GE Interlogix 60-807-95R Glassbreak Detector (SKU: 60-807-95R)', 'Bosch F220-B6 Detector Base (SKU: F220-B6)']
['Bosch F220-B6 Detector Base (SKU: F220-B6)', 'GE Interlogix 60-652-95R Carbon Monoxide Detector (SKU: 60-652-95R)', 'DSC WS4933 Carbon Monoxide Detector (SKU: WS4933)']
['GE Interlogix 60-652-95R Carbon Monoxide Detector (SKU: 60-652-95R)', 'Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2)', 'Bosch NDN-50022-A3 IP Camera (SKU: NDN-50022-A3)', 'Dahua DH-IPC-HDBW4431R-ZS IP Camera (SKU: DH-IPC-HDBW4431R-ZS)', 'Bosch B5512 Control Panel (SKU: B5512)']
['Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)', 'Pelco DSSRV2-040-US NVR (SKU: DSSRV2-040-US)', 'DSC WS4916 Smoke Detector (SKU: WS4916)', 

In [4]:
all_items = [item for txn in transactions for item in txn]

# Get unique items while preserving the order they first appear
seen = set()
unique_items = []
for item in all_items:
    if item not in seen:
        seen.add(item)
        unique_items.append(item)

# Create a DataFrame with sequential item_id
item_list_df = pd.DataFrame({
    "item_id": range(1, len(unique_items) + 1),
    "item": unique_items
})

# --- 5) Save item list to CSV ---
ITEM_LIST_PATH = "item_list.csv"
item_list_df.to_csv(ITEM_LIST_PATH, index=False)
print(f"\n✅ Saved item list to '{ITEM_LIST_PATH}' with {len(item_list_df)} unique items.")

# --- 6) Optional preview ---
print("\nItem list preview:")
print(item_list_df)


✅ Saved item list to 'item_list.csv' with 40 unique items.

Item list preview:
    item_id                                               item
0         1  Honeywell 5800CO Carbon Monoxide Detector (SKU...
1         2                Hanwha XRN-2010 NVR (SKU: XRN-2010)
2         3   Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)
3         4             Bosch B5512 Control Panel (SKU: B5512)
4         5     Pelco IMP1110-1ES IP Camera (SKU: IMP1110-1ES)
5         6  GE Interlogix 60-807-95R Glassbreak Detector (...
6         7         Bosch F220-B6 Detector Base (SKU: F220-B6)
7         8  GE Interlogix 60-652-95R Carbon Monoxide Detec...
8         9  DSC WS4933 Carbon Monoxide Detector (SKU: WS4933)
9        10         Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2)
10       11   Bosch NDN-50022-A3 IP Camera (SKU: NDN-50022-A3)
11       12  Dahua DH-IPC-HDBW4431R-ZS IP Camera (SKU: DH-I...
12       13       Pelco DSSRV2-040-US NVR (SKU: DSSRV2-040-US)
13       14            DSC WS4916 Smok

In [5]:
# association_rules = apriori(records, min_support=0.0045, min_confidence=0.001, min_length=2)
association_rules = apriori(transactions, min_support=0.005, min_confidence=0.01)
association_results = list(association_rules)



In [6]:
# print(association_results)

print(association_results[:3])

print(str(len(association_results)) + " items\n")

[RelationRecord(items=frozenset({'Axis M3046-V Network Camera (SKU: 0806-001)'}), support=0.08, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Axis M3046-V Network Camera (SKU: 0806-001)'}), confidence=0.08, lift=1.0)]), RelationRecord(items=frozenset({'Axis P3225-LVE Network Camera (SKU: 0935-001)'}), support=0.089, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Axis P3225-LVE Network Camera (SKU: 0935-001)'}), confidence=0.089, lift=1.0)]), RelationRecord(items=frozenset({'Axis P3367-VE Network Camera (SKU: 0407-001)'}), support=0.09, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Axis P3367-VE Network Camera (SKU: 0407-001)'}), confidence=0.09, lift=1.0)])]
673 items



In [7]:
for association in association_results:

    # first index of the inner list
    # Contains base item and add item

    items = [x for x in association[0]]
    if (len(association[0]) < 2):
        continue
    print(items)
    print("Support: " + str(association[1]))

    for rule in association[2]:
      # print("Rule: " + ",".join(items[:-1]) + " -> " + items[-1])
      LHS = set(rule[0])
      RHS = set(rule[1])
      print("--Rule: " + ",".join(LHS) + " -> " + ",".join(RHS))
      #second index of the inner list


      #third index of the list located at 0th
      #of the third index of the inner list
      print("  Confidence: " + str(association[2][0][2]))
      print("  Lift: " + str(association[2][0][3]))



print("DONE****")



['Axis M3046-V Network Camera (SKU: 0806-001)', 'Axis P3225-LVE Network Camera (SKU: 0935-001)']
Support: 0.009
--Rule: Axis M3046-V Network Camera (SKU: 0806-001) -> Axis P3225-LVE Network Camera (SKU: 0935-001)
  Confidence: 0.11249999999999999
  Lift: 1.2640449438202246
--Rule: Axis P3225-LVE Network Camera (SKU: 0935-001) -> Axis M3046-V Network Camera (SKU: 0806-001)
  Confidence: 0.11249999999999999
  Lift: 1.2640449438202246
['Axis M3046-V Network Camera (SKU: 0806-001)', 'Axis P3367-VE Network Camera (SKU: 0407-001)']
Support: 0.008
--Rule: Axis M3046-V Network Camera (SKU: 0806-001) -> Axis P3367-VE Network Camera (SKU: 0407-001)
  Confidence: 0.1
  Lift: 1.1111111111111112
--Rule: Axis P3367-VE Network Camera (SKU: 0407-001) -> Axis M3046-V Network Camera (SKU: 0806-001)
  Confidence: 0.1
  Lift: 1.1111111111111112
['Axis M3046-V Network Camera (SKU: 0806-001)', 'Bosch B5512 Control Panel (SKU: B5512)']
Support: 0.007
--Rule: Axis M3046-V Network Camera (SKU: 0806-001) -> Bos

In [8]:
def predict_next_items(transaction, association_results):
    """
    Predicts next probable items for a given transaction based on association rules.

    Args:
        transaction (list): A list of items in the current transaction.
        association_results (list): The list of association rules generated by apriori.
        min_support (float): Minimum support threshold for the rules.
        min_confidence (float): Minimum confidence threshold for the rules.

    Returns:
        list: A list of potential next items.
    """
    min_support = 0.01
    min_confidence = 0.01
    recommendations = []
    for association in association_results:
        antecedent = set(association[0])  # Items on the left-hand side of the rule
        support = association[1]
        if set(transaction).issubset(antecedent)and support >= min_support:
          predicted_items = set()
          # print(association)
          print("antecedent: " + str(antecedent))
          print("support:" + str(support))
          for rule in association[2]:
            LHS = set(rule[0])
            RHS = set(rule[1])
            confidence = rule[2]
            # if LHS.issubset(set(transaction)):
            if LHS == set(transaction):
            # Add the consequent items to the predicted items, excluding items already in the transaction
              predicted_items.update(RHS - set(transaction))
              print("RHS: " + str(RHS))
              print("confidence: " + str(confidence))
              recommendations.append({"items": list(predicted_items), "confidence":confidence})

    # Sort the recommendations by confidence in descending order
    recommendations.sort(key=lambda x: x["confidence"], reverse=True)
    return recommendations



In [9]:
# my_transaction = test_data.iloc[0].dropna().tolist() # Select the first row and convert to a list of non-null values
my_transaction = ['Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)']
# my_transaction = ['Organic Pink Lemonade Bunny Fruit Snacks', 'Crunch Chocolate Peanut Butter Granola Bar']
print(f"Test data transaction: {my_transaction}")


Test data transaction: ['Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)']


In [10]:
# Predict the next items
next_items = predict_next_items(my_transaction, association_results)

print(f"Potential next items: {next_items}")

antecedent: {'Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)'}
support:0.083
antecedent: {'Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2)', 'Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)'}
support:0.014
RHS: {'Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2)'}
confidence: 0.1686746987951807
antecedent: {'GE Interlogix 60-652-95R Carbon Monoxide Detector (SKU: 60-652-95R)', 'Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)'}
support:0.011
RHS: {'GE Interlogix 60-652-95R Carbon Monoxide Detector (SKU: 60-652-95R)'}
confidence: 0.1325301204819277
antecedent: {'Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)', 'Hikvision DS-2CD2385FWD-I IP Camera (SKU: DS-2CD2385FWD-I)'}
support:0.011
RHS: {'Hikvision DS-2CD2385FWD-I IP Camera (SKU: DS-2CD2385FWD-I)'}
confidence: 0.1325301204819277
antecedent: {'Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)', 'Hikvision DS-7608NI-I2/8P NVR (SKU: DS-7608NI-I2/8P)'}
support:0.012
RHS: {'Hikvision DS-7608NI-I2/8P NVR (SKU: DS-7608NI-I2/8P)'}
confidence: 0.1445

Part II. Build Gradio UI for demo

In [11]:
!pip install -qU gradio==4.* pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 11.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
yfinance 0.2.66 requires websockets>=13.0, but you have websockets 12.0 which is incompatible.
google-genai 1.49.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 1

In [12]:
import ast
import os
import pandas as pd
import gradio as gr

# -------------------------------
# 1) Load inventory (static list)
# -------------------------------
ITEMS_CSV = "item_list.csv"  # columns: item_id, item
item_list_df = pd.read_csv(ITEMS_CSV)
inventory = item_list_df["item"].astype(str).tolist()
print(f"Loaded {len(inventory)} items from '{ITEMS_CSV}'.")

# -------------------------------
# 3) Recommendation function
# -------------------------------
def recommend(selected_items):
    """
    Given a list of selected items (subset of inventory),
    return recommendations ranked by confidence (desc),
    tie-breaking by lift then support if available.
    """
    selected = list(selected_items or [])
    if not selected:
        # As per requirement, return an empty list (no recs without input)
        return pd.DataFrame(columns=["item", "confidence"])

    # Filter rules where rule.antecedents ⊆ selected
    recommendations = predict_next_items(selected_items, association_results)
    if not recommendations:
        return pd.DataFrame(columns=["item", "confidence"])

    df = pd.DataFrame([
    {
        "recommended items": ", ".join(sorted(r["items"])),  # turn set/list into a readable string
        "confidence": r["confidence"]
    }
    for r in recommendations
])
    return df

# -------------------------------
# 4) Gradio UI
# -------------------------------
with gr.Blocks(title="ARM Recommender") as demo:
    gr.Markdown("# Association Rule Recommender")
    gr.Markdown(
        "Select items from your inventory. The recommender returns items implied by your selection via association rules, "
        "ranked by **confidence** (ties by lift/support). If no rule applies, you’ll get an empty list."
    )

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Inventory (static)")
            inv_df = gr.Dataframe(
                value=item_list_df,
                wrap=True,
                interactive=False,
                height=300
            )
            gr.Markdown("### Choose items")
            item_selector = gr.CheckboxGroup(
                choices=inventory,
                label="Your items",
                info="Pick one or more items from inventory."
            )
            run_btn = gr.Button("Get Recommendations", variant="primary")

        with gr.Column(scale=1):
            gr.Markdown("### Recommendations")
            recs_out = gr.Dataframe(
                headers=["item", "confidence"],
                interactive=False,
                height=400
            )

    # Wire up
    run_btn.click(fn=recommend, inputs=item_selector, outputs=recs_out)




Loaded 40 items from 'item_list.csv'.


In [ ]:
# Launch (debug=True for debugging mode)
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://84d9459048ed68439a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


antecedent: {'Honeywell 5800CO Carbon Monoxide Detector (SKU: 5800CO)'}
support:0.075
antecedent: {'DSC WS4916 Smoke Detector (SKU: WS4916)', 'Honeywell 5800CO Carbon Monoxide Detector (SKU: 5800CO)'}
support:0.015
RHS: {'DSC WS4916 Smoke Detector (SKU: WS4916)'}
confidence: 0.2
antecedent: {'Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)'}
support:0.083
antecedent: {'Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2)', 'Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)'}
support:0.014
RHS: {'Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2)'}
confidence: 0.1686746987951807
antecedent: {'GE Interlogix 60-652-95R Carbon Monoxide Detector (SKU: 60-652-95R)', 'Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)'}
support:0.011
RHS: {'GE Interlogix 60-652-95R Carbon Monoxide Detector (SKU: 60-652-95R)'}
confidence: 0.1325301204819277
antecedent: {'Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)', 'Hikvision DS-2CD2385FWD-I IP Camera (SKU: DS-2CD2385FWD-I)'}
support:0.011
RHS: {'Hikvision DS-2CD2385FWD-I I